In [1]:
train_size = 47500

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
from sklearn.model_selection import cross_validate
%matplotlib inline

In [3]:
np.set_printoptions(precision=9)
np.set_printoptions(linewidth=np.inf)

In [5]:
X = np.load('../../X_train.npz')['arr_0']
Y = np.load('../../Y_train.npz')['arr_0']
TX = np.load('../../X_test.npz')['arr_0']

In [6]:
rd =  np.random.permutation(47500)
X, Y = X[rd], Y[rd]

In [7]:
XTX = np.concatenate((X, TX), axis=0)

In [8]:
XTX_square = XTX ** 2
XTX_cumsum = np.concatenate((np.cumsum(XTX[:, :5000], axis=1), np.cumsum(XTX[:, 5000:], axis=1)), axis=1)
XTX_all = np.concatenate((XTX, XTX_square, XTX_cumsum), axis=1)

In [11]:
XTX_all.shape

(50000, 30000)

In [10]:
# calculate type 1 error
def err1(y, y_pred):
    return np.sum(1.0 * np.abs(y_pred - y)) / len(y_pred)

# calculate type 2 errr
def err2(y, y_pred):
    return np.sum(np.abs(y_pred - y) / y) / len(y_pred)

In [ ]:
y_id = 0

gbm0 = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000,
                        n_jobs=-1)

t = time.time()
gbm0.fit(XTX_all[:42500], Y[:42500, y_id], 
        eval_set=[(XTX_all[42500:45000], Y[42500:45000, y_id])], 
        eval_metric='l1', 
        early_stopping_rounds=5)
print(time.time() - t)

print('in:')
print(300.0 * err1(gbm0.predict(XTX_all[:42500]), Y[:42500, y_id]))
print(err2(gbm0.predict(XTX_all[:42500]), Y[:42500, y_id]))

print('val:')
print(300.0 * err1(gbm0.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))
print(err2(gbm0.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))

print('test:')
print(300.0 * err1(gbm0.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))
print(err2(gbm0.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))

ty0 = gbm0.predict(XTX_all[47500:])

In [ ]:
y_id = 1

gbm1 = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000,
                        n_jobs=-1)

t = time.time()
gbm1.fit(XTX_all[:42500], Y[:42500, y_id], 
        eval_set=[(XTX_all[42500:45000], Y[42500:45000, y_id])], 
        eval_metric='l1', 
        early_stopping_rounds=5)
print(time.time() - t)

print('in:')
print(1.0 * err1(gbm1.predict(XTX_all[:42500]), Y[:42500, y_id]))
print(err2(gbm1.predict(XTX_all[:42500]), Y[:42500, y_id]))

print('val:')
print(1.0 * err1(gbm1.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))
print(err2(gbm1.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))

print('test:')
print(1.0 * err1(gbm1.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))
print(err2(gbm1.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))

ty1 = gbm1.predict(XTX_all[47500:])

In [ ]:
y_id = 2

gbm2 = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=1000,
                        n_jobs=-1)

t = time.time()
gbm2.fit(XTX_all[:42500], Y[:42500, y_id], 
        eval_set=[(XTX_all[42500:45000], Y[42500:45000, y_id])], 
        eval_metric='l1', 
        early_stopping_rounds=5)
print(time.time() - t)

print('in:')
print(200.0 * err1(gbm2.predict(XTX_all[:42500]), Y[:42500, y_id]))
print(err2(gbm2.predict(XTX_all[:42500]), Y[:42500, y_id]))

print('val:')
print(200.0 * err1(gbm2.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))
print(err2(gbm2.predict(XTX_all[42500:45000]), Y[42500:45000, y_id]))

print('test:')
print(200.0 * err1(gbm2.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))
print(err2(gbm2.predict(XTX_all[45000:47500]), Y[45000:47500, y_id]))

ty2 = gbm2.predict(XTX_all[47500:])

In [ ]:
ty = np.array([ty0, ty1, ty2]).transpose()
print(ty.shape)
df = pd.DataFrame(ty)
df.to_csv('y_test.csv', index=False, header=False)